In [6]:
import pandas as pd
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim

I read the csv file with all Toronto Postcode coordinates and convert it to a pandas dataframe

In [18]:
toronto = pd.read_csv('toronto_co.csv')

toronto.head()

I find the coordinates of Toronto

In [20]:
address = 'Toronto, Canada'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_x = location.latitude
longitude_y = location.longitude
print('The geographical coordinates of Toronto are {}, {}.'.format(latitude_x,longitude_y))


/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


The geographical coordinates of Toronto are 43.653963, -79.387207.


I create the map of Toronto

In [21]:
map_toronto = folium.Map(location=[latitude_x,longitude_y], zoom_start=10)
map_toronto

I insert the markers with the Neighbours

In [25]:
for lat, lng, bor, nei in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'],toronto['Neighbourhood']):
    
    label = '{}, {}'.format(nei, bor)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'red',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity=0.7,
        parse_html = False).add_to(map_toronto)

map_toronto